In [33]:
from __future__ import print_function
import shap
import numpy as np
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import datetime
#To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
import os

In [34]:
print(tf.version)

<module 'tensorflow._api.v2.version' from 'C:\\Users\\ayoub\\.conda\\envs\\shap\\lib\\site-packages\\tensorflow\\_api\\v2\\version\\__init__.py'>


In [35]:
# Proprocessing the test and training set
train_datagen = ImageDataGenerator(rescale = 1./255, #pixelwaarden normaliseren
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('C:/Users/ayoub/OneDrive/TMM/Stage fase 3/Arinti/FindWaldo/FindWaldo/Scripts/images/training',
                                                 target_size = (64, 64),
                                                 batch_size = 32, #hoeveel foto's per keer langs neural netwerk laat passeren
                                                 class_mode = 'binary')
#fit from generator
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('C:/Users/ayoub/OneDrive/TMM/Stage fase 3/Arinti/FindWaldo/FindWaldo/Scripts/images/testing',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 3047 images belonging to 2 classes.
Found 1309 images belonging to 2 classes.


In [36]:
%load_ext tensorboard

# Initialising the CNN
cnn = tf.keras.models.Sequential() # layers gaan op elkaar volgen

# Add convolution layer
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

# Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Add convolution layer
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))

# Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2))

# Flattening
cnn.add(tf.keras.layers.Flatten()) # full connection maken (zorgt voor features)

# Full connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Output layer
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) # sigmoid = input transformeren naar 0-1

#When training with Keras's Model.fit(), adding the tf.keras.callbacks.
#TensorBoard callback ensures that logs are created and stored. 
#Additionally, enable histogram computation every epoch with histogram_freq=1 (this is off by default)
log_dir = "/TensorBoard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


# Compile CNN
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

cnn.fit(x = training_set, validation_data = test_set, epochs = 1)

%tensorboard --logdir TensorBoard/fit
%tensorboard --logdir TensorBoard/scalars

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
96/96 [==============================] - 11s 103ms/step - loss: 0.0808 - accuracy: 0.9797 - val_loss: 0.0452 - val_accuracy: 0.9931


In [37]:
# Making a prediction
test_image = image.load_img('C:/Users/ayoub/OneDrive/TMM/Stage fase 3/Arinti/FindWaldo/FindWaldo/Scripts/images/testing/waldo/12_2_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  print("Tested negative for Waldo :'(")
else:
  print('Tested positive for Waldo! :D')

Tested positive for Waldo! :D


In [38]:
x_train, y_train = next(training_set)
x_test, y_test = next(test_set)
#model.fit(x = training_set, validation_data = test_set, epochs = 1)
score = cnn.evaluate(test_set, verbose=0) 
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.04520579054951668
Test accuracy: 0.9931245446205139


In [39]:
# select a set of background examples to take an expectation over
background = x_train[np.random.choice(x_train.shape[0], 100, replace=True)]

In [40]:
# explain predictions of the model (named cnn in this case) on three images
#e = shap.DeepExplainer(cnn, background)
e = shap.DeepExplainer((cnn.layers[0].input, cnn.layers[-1].output), background)

In [41]:
from tensorflow.python.keras import backend
graph = backend.get_graph()
print(graph)
#return explainer.model_output.graph

FuncGraph(name=keras_graph, id=2675238023072)


In [42]:
shap_values = e.shap_values(x_test[1:5][1:5])
print(shap_values)

[array([[[[-8.96248631e-10, -2.62851376e-08, -1.31589999e-07],
         [-5.12888612e-08, -3.06163571e-07,  1.11735808e-07],
         [ 5.60603894e-07, -4.74160544e-07, -2.67155229e-07],
         ...,
         [-7.04347300e-08, -4.95301009e-08,  8.86589007e-08],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

        [[ 5.83678776e-08, -9.80215997e-09, -1.18239228e-07],
         [-3.36068808e-07,  1.54505742e-07, -2.24200267e-08],
         [-6.09566513e-08,  2.97451635e-07, -2.47460665e-08],
         ...,
         [-6.28890149e-08,  2.02361683e-08,  1.06668043e-08],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

        [[ 4.78326626e-07,  1.43323788e-07, -7.09812639e-07],
         [ 4.78596917e-07, -1.61043857e-07,  8.44553085e-07],
         [ 3.37315868e-07,  8.00890291e-07,  4.55581215e-07],
         ...,
         [-4.15206188e-